In [10]:
def compress_data(data):

    for eachtype in set(data.dtypes.values):
        ##檢查數據的類型
        if 'int' in str(eachtype):
            ## 對每列進行轉換類型的轉換
            for i in data.select_dtypes(eachtype).columns.values:
                if data[i].min() < 0:
                    data[i] = pd.to_numeric(data[i],downcast='signed')
                else:
                    data[i] = pd.to_numeric(data[i],downcast='unsigned')      
        elif 'float' in str(eachtype):
            for i in data.select_dtypes(eachtype).columns.values:   
                data[i] = pd.to_numeric(data[i],downcast='float')
        elif 'object' in str(eachtype):
            for i in data.select_dtypes(eachtype).columns.values: 
                data[i] = trainData7[i].astype('category')
    return data
 
user_log = pd.read_csv('user_log_format1.csv')
print('Before compress_data:\n',user_log.info())
user_log = compressData(user_log)
print('After compress_data:\n',user_log.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      uint32 
 1   item_id      uint32 
 2   cat_id       uint16 
 3   seller_id    uint16 
 4   brand_id     float32
 5   time_stamp   uint16 
 6   action_type  uint8  
dtypes: float32(1), uint16(3), uint32(2), uint8(1)
memory usage: 995.2 MB
Before compressed:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      uint32 
 1   item_id      uint32 
 2   cat_id       uint16 
 3   seller_id    uint16 
 4   brand_id     float32
 5   time_stamp   uint16 
 6   action_type  uint8  
dtypes: float32(1), uint16(3), uint32(2), uint8(1)
memory usage: 995.2 MB
After compressed:
 None


In [41]:
user_info = pd.read_csv('user_info_format1.csv')


In [14]:
#將所有缺失值的索引都找出來
missing_value = user_log[user_log.brand_id.isnull()].index
#將seller_id合併並且選出每個seller_id中brand_id的眾數代表他可能最常逛的品牌作為填充
seller_mode = user_log.groupby(['seller_id']).apply(lambda x:x.brand_id.mode()[0]).reset_index()



In [22]:
#將缺失數據挑選出來
missing_pickup = user_log.loc[missing_value]
#向左合併數據已missing_pickup為主並且以seller_mode中索取到的眾數
missing_pickup = pd.merge(missing_pickup,seller_mode ,how='left',on=['seller_id']).astype('float32')

missing_pickup = missing_value
#選出缺失值並將用剛剛用好的眾數得值取代掉
user_log.loc[missing_value,'brand_id'] = missing_pickup


In [ ]:
#填充好後節省空間將多餘數值刪除
del pickUP,sellerMode,missingIndex

In [42]:
#用戶特徵
user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

In [43]:
#將用戶特徵進行填充
#年齡用中位數
user_info.age_range.fillna(user_info.age_range.median(),inplace=True)
#性別使用眾數
user_info.gender.fillna(user_info.gender.mode()[0],inplace=True)

In [44]:
user_info.isnull().sum()

user_id      0
age_range    0
gender       0
dtype: int64

In [45]:
#對age 與 gender 進行 get_dummy
df_age= pd.get_dummies(user_info.age_range,prefix='age')
df_sex = pd.get_dummies(user_info.gender)
#對做好的性別啞變量進行更名
df_sex.rename(columns={0:'female',1:'male',2:'unknown'},inplace=True)

In [46]:
#對剛剛創建好的數據與user原始數據進行合併
user_info = pd.concat([user_info.user_id,df_age,df_sex],axis=1)

In [51]:
#用戶行為提取
action_total = user_log[['user_id','action_type']]

In [53]:
#進行為進行啞變量
df=pd.get_dummies(action_total['action_type'],prefix='user_action_total',ax_index=False)

In [58]:
#對剛剛建立好的啞變量進行根據每個人進行行為次數的加總
#ax_index在消除分組鍵的同時會自動生成一個索引
action_total = pd.concat([action_total.user_id,df],axis=1).groupby(['user_id'],as_index=False).sum()
action_total['user_action_total']=action_total['user_action_total_0']+action_total['user_action_total_1']+action_total['user_action_total_2']+action_total['user_action_total_3']
action_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              424170 non-null  uint64 
 1   user_action_total_0  424170 non-null  float64
 2   user_action_total_1  424170 non-null  float64
 3   user_action_total_2  424170 non-null  float64
 4   user_action_total_3  424170 non-null  float64
 5   user_action_total    424170 non-null  float64
dtypes: float64(5), uint64(1)
memory usage: 22.7 MB


In [69]:
#用戶互動次數在所有用戶中的類別

#用戶互動次數佔所有用戶互動次數的比例

print('所有行為次數: ' +str(user_log.shape[0]))
print('所有用戶數目: ' +str(user_log['user_id'].nunique()))
print('所有用戶平均行為次數：'+str(user_log.shape[0]/user_log['user_id'].nunique()))
action_total['user_total_action_ratio']=action_total['user_action_total']/user_log.shape[0]
#user_log.shape[0]/userLog['user_id'].nunique() 代表平均每個人要多少次行為
#user_total_action_diff 代表每個人總行為的次數與整體每個人平均行為的次數可以推斷這個人對於網站的偏好度是否高
action_total['user_total_action_diff'] = action_total['user_action_total']-user_log.shape[0]/user_log['user_id'].nunique()

所有行為次數: 54925330
所有用戶數目: 424170
所有用戶平均行為次數：129.48895490015795


In [78]:
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


**用戶點擊次數在所有用戶點擊次數中的一部分**

●用戶點擊次數佔所有用戶點擊次數的比例

●用戶點擊次數與用戶平均點擊次數的差值（絕對差值，相對差值）

●用戶點擊次數在所有用戶點擊次數中的分位數

●用戶點擊次數在所有用戶點擊次數中的排名

In [99]:
#用戶點擊次數在所有用戶點擊次數中的一部分

#用戶點擊次數佔所有用戶點擊次數的比例

print('所有用戶點擊次數:'+str(user_log[user_log.action_type==0].shape[0]))
action_total['user_click_rate'] = action_total['user_action_total_0']/ user_log[user_log.action_type==0].shape[0]
print('用戶平均點擊次數:' +str(user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()))
action_total['user_click_diff']=action_total['user_action_total_0']-user_log[user_log.action_type==0].shape[0]/user_log['user_id'].nunique()


所有用戶點擊次數:48550713
用戶平均點擊次數:114.46050640073555


In [100]:
action_total

,user_id,user_action_total_0,user_action_total_1,user_action_total_2,user_action_total_3,user_action_total,user_total_action_ratio,user_total_action_diff,user_click_rate,user_click_diff,user_addtocart_rate,user_addtocart_diff
0,1,27.0,0.0,6.0,0.0,33.0,6.008157e-07,-96.488955,5.561195e-07,-87.460506,0.0,0.0
1,2,47.0,0.0,14.0,2.0,63.0,1.147012e-06,-66.488955,9.680599e-07,-67.460506,0.0,0.0
2,3,63.0,0.0,4.0,1.0,68.0,1.238044e-06,-61.488955,1.297612e-06,-51.460506,0.0,0.0
3,4,49.0,0.0,1.0,0.0,50.0,9.103268e-07,-79.488955,1.009254e-06,-65.460506,0.0,0.0
4,5,150.0,0.0,13.0,10.0,173.0,3.149731e-06,43.511045,3.089553e-06,35.539494,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
424165,424166,79.0,0.0,11.0,0.0,90.0,1.638588e-06,-39.488955,1.627165e-06,-35.460506,0.0,0.0
424166,424167,28.0,0.0,6.0,1.0,35.0,6.372288e-07,-94.488955,5.767166e-07,-86.460506,0.0,0.0
424167,424168,216.0,0.0,6.0,1.0,223.0,4.060058e-06,93.511045,4.448956e-06,101.539494,0.0,0.0
424168,424169,277.0,0.0,17.0,3.0,297.0,5.407341e-06,167.511045,5.705375e-06,162.539494,0.0,0.0


In [101]:
print('所有用戶加入購物次：'+str(user_log[user_log.action_type==1].shape[0]))
action_total['user_addtocart_rate']=action_total['user_action_total_1']/user_log[user_log.action_type==1].shape[0]
print('用戶平均加入購物車次數：'+str(user_log[user_log.action_type==1].shape[0]/user_log['user_id'].nunique()))
action_total['user_addtocart_diff']=action_total['user_action_total_1']-user_log[user_log.action_type==1].shape[0]/user_log['user_id'].nunique()

所有用戶加入購物次：76750
用戶平均加入購物車次數：0.18094160360232925


In [103]:
#查看一下不等於0的結果
action_total[action_total.user_action_total_1!=0].head()

,user_id,user_action_total_0,user_action_total_1,user_action_total_2,user_action_total_3,user_action_total,user_total_action_ratio,user_total_action_diff,user_click_rate,user_click_diff,user_addtocart_rate,user_addtocart_diff
17,18,106.0,3.0,1.0,0.0,110.0,2.002719e-06,-19.488955,2.183284e-06,-8.460506,0.000039,2.819058
22,23,165.0,3.0,6.0,0.0,174.0,3.167937e-06,44.511045,3.398508e-06,50.539494,0.000039,2.819058
40,41,77.0,1.0,4.0,1.0,83.0,1.511142e-06,-46.488955,1.585971e-06,-37.460506,0.000013,0.819058
42,43,26.0,1.0,2.0,0.0,29.0,5.279895e-07,-100.488955,5.355225e-07,-88.460506,0.000013,0.819058
45,46,176.0,1.0,11.0,0.0,188.0,3.422829e-06,58.511045,3.625075e-06,61.539494,0.000013,0.819058


In [104]:
#用戶購買次數在所有用戶購買次數中的一部分

#用戶購買次數佔所有用戶購買次數的比例
print('所有用戶購買次數： '+str(user_log[user_log.action_type==2].shape[0]))
action_total['user_purchase_rate']=action_total['user_action_total_2'] / user_log[user_log.action_type==2].shape[0]
print('平均每人購買次數： '+str(user_log[user_log.action_type==2].shape[0]/user_log['user_id'].nunique()))
action_total['user_purchase_diff'] = action_total['user_action_total_2'] -user_log[user_log.action_type==2].shape[0]/user_log['user_id'].nunique()


所有用戶購買次數： 3292144
平均每人購買次數： 7.761378692505364


In [105]:
#用戶收藏次數在所有用戶收藏次數中的一部分

#用戶收藏次數佔所有用戶收藏次數的比例
print('所有收藏購買次數： '+str(user_log[user_log.action_type==3].shape[0]))
action_total['user_save_rate']=action_total['user_action_total_3'] / user_log[user_log.action_type==3].shape[0]
print('平均每人收藏次數： '+str(user_log[user_log.action_type==3].shape[0]/user_log['user_id'].nunique()))
action_total['user_save_diff'] = action_total['user_action_total_3'] -user_log[user_log.action_type==3].shape[0]/user_log['user_id'].nunique()


所有收藏購買次數： 3005723
平均每人收藏次數： 7.086128203314709




●用戶加入購物車次數佔用戶總交互次數的比例

●用戶購買次數佔用戶總互動次數的比例

●用戶收藏次數佔用戶總互動次數的比例

In [110]:
#統計用戶不同行為的習慣（用戶內部）

#用戶點擊次數佔用戶總互動次數的比例
#統計用戶點擊、加入購物車、收藏、購買次數佔整體行為次數的比例
action_total['click_action_rate']=action_total['user_action_total_0'] / action_total['user_action_total']
action_total['addtocart_action_rate']=action_total['user_action_total_1'] / action_total['user_action_total']
action_total['purchase_action_rate']=action_total['user_action_total_2'] / action_total['user_action_total']
action_total['save_action_rate']=action_total['user_action_total_3'] / action_total['user_action_total']


**統計用戶的點擊，加入購物車，收藏者的購買轉化率**

●用戶點擊轉化率

●用戶加入購物車購買轉化率

●用戶收藏轉化率

In [139]:
#統計用戶的點擊，加入購物車，收藏者的購買轉化率

#用戶點擊轉化率

#用戶加入購物車購買轉化率

●#用戶收藏轉化率


#購買/點擊
action_total['user_pruchase_ratio'] = action_total['user_action_total_2'] / action_total['user_action_total_0']
#加入購物車/點擊
action_total['user_addtocart_ratio'] = action_total['user_action_total_1'] / action_total['user_action_total_0']
#收藏/點擊
action_total['user_save_ratio'] = action_total['user_action_total_3'] / action_total['user_action_total_0']

In [144]:
#用戶互動的時間信息（按天）
#用戶互動的總天數，與所有用戶平均互動總天數的比較
#用戶每個月的交互天數，與所有用戶平均每個月的交互天數的比較
#用戶月互動天數的變化量，與所有用戶平均月交互天數變化量的比較
#算出每個人有任何行動的天數的次數
day_cnt = user_log.groupby(['user_id'])['time_stamp'].nunique()
#再將這些次去剪掉平均整體每個人，可以知道用戶對於網站是否常來購買
days_cnt_diff =day_cnt - user_log.groupby(['user_id'])['time_stamp'].nunique().mean()

In [149]:
#用戶互動的時間信息（按次數）**
#用戶互動的總天數，與所有用戶平均互動總天數的比較
#用戶每個月的交互天數，與所有用戶平均每個月的交互天數的比較
#用戶月互動天數的變化量，與所有用戶平均月交互天數變化量的比較
#先剛剛剛做出來所有浮點數類型的維度進行標準化避免數字大小差異過大造成後續模型計算上的偏誤
num_col = action_total.columns[action_total.dtypes == 'float64']
num_col_mean = action_total.loc[:,num_col].mean()
num_col_std  = action_total.loc[:,num_col].std()
action_total.loc[:,num_col]=(action_total.loc[:,num_col] - num_col_mean) / num_col_std


In [152]:
#將上述做好的資料進行串接
user_info = pd.merge(user_info,action_total,how='left',on=['user_id'])


In [160]:
print(user_log.time_stamp.max())
print(user_log.time_stamp.min())

1112
511


In [178]:
#用戶6個月中做出行為的商品數量
item_count = user_log.groupby(['user_id'])['item_id'].nunique()
#用戶6個月中做出行為的種類數量
cat_count = user_log.groupby(['user_id'])['cat_id'].nunique()
#用戶6個月中做出行為的店鋪數量
seller_count = user_log.groupby(['user_id'])['seller_id'].nunique()
#用戶6個月中做出行為的品牌數量
brand_count = user_log.groupby(['user_id'])['brand_id'].nunique()
#用戶6個月中做出行為的行為數量
days_count = user_log.groupby(['user_id'])['time_stamp'].nunique()

allcount = pd.concat([item_count,cat_count],axis=1)
allcount = pd.concat([allcount,seller_count],axis=1)
allcount = pd.concat([allcount,brand_count],axis=1)
allcount = pd.concat([allcount,days_count],axis=1)


In [179]:
allcount.rename(columns={'item_id':'item_count','cat_id':'cat_count','seller_id':'seller_count','brand_id':'brand_count','time_stamp':'active_days'},inplace=True)


In [180]:
allcount.reset_index()

,user_id,item_count,cat_count,seller_count,brand_count,active_days
0,1,12,6,9,9,5
1,2,43,14,14,15,9
2,3,45,19,23,22,13
3,4,28,13,12,12,10
4,5,87,40,56,60,30
...,...,...,...,...,...,...
424165,424166,48,12,22,23,12
424166,424167,15,9,12,9,8
424167,424168,160,48,112,97,55
424168,424169,176,46,102,103,34


In [181]:
numeric_cols = allcount.columns[allcount.dtypes == 'int64']
print(numeric_cols)
numeric_col_means = allcount.loc[:, numeric_cols].mean()
numeric_col_std = allcount.loc[:, numeric_cols].std()
allcount.loc[:, numeric_cols] = (allcount.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std
allcount.head(5)

Index(['item_count', 'cat_count', 'seller_count', 'brand_count',
       'active_days'],
      dtype='object')


,item_count,cat_count,seller_count,brand_count,active_days
user_id,,,,,
1,-0.619293,-0.985184,-0.713461,-0.725656,-0.839753
2,-0.315566,-0.507574,-0.565709,-0.544117,-0.557186
3,-0.295971,-0.209068,-0.299757,-0.332322,-0.274619
4,-0.462530,-0.567275,-0.624810,-0.634886,-0.486544
5,0.115530,1.044659,0.675404,0.817425,0.926290


In [171]:
num_col

Index(['item_count', 'cat_count', 'seller_count', 'brand_count',
       'active_days'],
      dtype='object')

In [197]:
#統計在雙11之前是否就有消費者有重複購買的情況發生
#先選取user、seller、時間、與行為的內容
repurchase = user_log[['user_id','seller_id','time_stamp','action_type']]
#篩選出行為是購買的並且購買的時間要小於 1111這天
repurchase = repurchase[(repurchase.action_type==2) &(repurchase.time_stamp<1111)]


In [198]:
#因為可能會有一天購買多筆的情況出現所以將重複的刪除
repurchase.drop_duplicates(inplace=True)
#因主要要看是否有多少消費者已經購買過某一個店家重複的商品，所以groupby 的維度為user_id、seller_id並
repurchase = repurchase.groupby(['user_id','seller_id'])['time_stamp'].count().reset_index()

In [199]:
#選取大於1的資料
repurchase = repurchase[repurchase.time_stamp>1]
#選出使用者ID並且購買次數大於二的店家
repurchase = repurchase.groupby(['user_id'])['seller_id'].count().reset_index()
#將表格重新命名
repurchase.rename(columns={'seller_id':'repurchase_count'},inplace=True)

In [201]:
#對數值型特徵進行標準化
numeric_col=repurchase.columns[repurchase.dtypes=='int64']
print(numeric_col)
numeric_col_mean = repurchase.loc[:,numeric_col].mean()
numeric_col_std = repurchase.loc[:,numeric_col].std()
repurchase.loc[:,numeric_col]=(repurchase.loc[:,numeric_col]-numeric_col_mean)/numeric_col_std
repurchase

Index(['repurchase_count'], dtype='object')


,user_id,repurchase_count
0,6,-0.331954
1,8,0.590256
2,14,2.434676
3,21,2.434676
4,25,-0.331954
...,...,...
74760,424131,-0.331954
74761,424133,-0.331954
74762,424134,-0.331954
74763,424153,-0.331954


In [202]:
#將資料合併到原本的user_info上
user_info = pd.merge(user_info,repurchase,how='left',on=['user_id'])
#假如遇到沒有重複購買的使用者用"0" 進行填充
user_info.repurchase_count.fillna(0,inplace=True)
#因為可能在一家店面重複購買多次所以將所有>=1的數字都調整為1
user_info['repurchase_count']=user_info['repurchase_count'].apply(lambda x:1 if x!=0 else 0)


In [204]:
del repurchase

**用戶的活躍程度的變化**

●用戶每個月點擊的次數

●用戶每個月加購的次數

●用戶每個月購買的次數

●用戶每個月收藏的次數

●用戶每個月互動的總次數

In [206]:
monthly_action_count = user_log[['user_id','time_stamp','action_type']]
result = list()
for i in range(5,12):
    #建構開始日
    start = int(str(i)+'00')
    #建構結束日
    end = int(str(i)+'30')
    #拿與第I個月份的數據
    month_exam = monthly_action_count[(monthly_action_count.time_stamp>=start) & (monthly_action_count.time_stamp<=end)]
    #對拿取出來的月份進行啞編碼
    df = pd.get_dummies(month_exam['action_type'],prefix='%d_action'%i)
    df['Month'+str(i)+'_action'] = df['Month'+str(i)+'_action_0']+df['Month'+str(i)+'_action_1']+df['Month'+str(i)+'_action_2']+df['Month'+str(i)+'_action_3']
    # 將example 的 時段的數據設定為月份值
    month_exam.loc[:,'time_stamp'] = month_exam.time_stamp.apply(lambda x:int(str(x)[0]) if len(str(x)) == 3 else int(str(x)[:2]))
    result.append(pd.concat([month_exam,df],axis=1).groupby(['user_id','time_stamp'],as_index=False).sum())
    
for i in range(0,7):
    user_info = pd.merge(user_info,result[i],how='left',on=['user_id'])
    user_info.fillna(0,inplace=True)

C:\Users\austin\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [211]:
#將多餘的action_type 與 time_stamp刪除
for i in ['time_stamp_x','action_type_x','time_stamp_y','action_type_y','time_stamp','action_type']:
    del user_info[i]
for i in range(5,12):
    user_info['Month'+str(i)+'_action'] = user_info['Month'+str(i)+'_action_0']+user_info['Month'+str(i)+'_action_1']+user_info['Month'+str(i)+'_action_2']+user_info['Month'+str(i)+'_action_3']



In [213]:
import pickle
import os

In [214]:
user_file_path = 'user_info_features.pkl'
pickle.dump(user_info,open(user_file_path,'wb'))

In [221]:
#讀取特徵
user_file_path='user_info_features.pkl'
if os.path.exists(user_file_path):
    user_info = pickle.load(open(user_file_path,'rb'))
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424170 entries, 0 to 424169
Data columns (total 72 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   user_id                    424170 non-null  int64  
 1   age_0.0                    424170 non-null  uint8  
 2   age_1.0                    424170 non-null  uint8  
 3   age_2.0                    424170 non-null  uint8  
 4   age_3.0                    424170 non-null  uint8  
 5   age_4.0                    424170 non-null  uint8  
 6   age_5.0                    424170 non-null  uint8  
 7   age_6.0                    424170 non-null  uint8  
 8   age_7.0                    424170 non-null  uint8  
 9   age_8.0                    424170 non-null  uint8  
 10  female                     424170 non-null  uint8  
 11  male                       424170 non-null  uint8  
 12  unknown                    424170 non-null  uint8  
 13  user_action_total_0        42

In [222]:
#統計每個用戶的商品，商品的類別、這些商品品牌總數，並合併成表格形式如上面用戶特徵一樣，並將這些特徵命好名稱
item_count = user_log[['seller_id','item_id']].groupby(['seller_id'])['item_id'].nunique().reset_index()
cat_count = user_log[['seller_id','cat_id']].groupby(['seller_id'])['cat_id'].nunique().reset_index()
brand_count = user_log[['seller_id','brand_id']].groupby(['seller_id'])['brand_id'].nunique().reset_index()
item_count.rename(columns={'item_id':'item_count'},inplace=True)
cat_count.rename(columns={'cat_id':'cat_count'},inplace=True)
brand_count.rename(columns={'brand_id':'brand_count'},inplace=True)


In [250]:
#商品在雙11之前的重複買家的數量
#第一步選取雙11以前的數據並且將action_type鎖定在購買過
repurchase_count = user_log[(user_log.time_stamp < 1111) &(user_log.action_type==2)]
#將數據根據每個銷售Id呈列出有哪些user_id 使用並將其轉呈dataframe 的形式
repurchase_count = repurchase_count.groupby(['seller_id'])['user_id'].value_counts().to_frame()
#將數據名稱重新命名
repurchase_count.rename(columns={'user_id':'user_buy_count'},inplace=True)
#目前看來像是樞紐分析表並沒有，每列的index 給予每列相對應的代號
repurchase_count.reset_index(inplace=True)
#因為要計算那些人重複購買篩選出買過大於1次的
repurchase_count = repurchase_count[repurchase_count.user_buy_count >1]
#計算出每個seller_id 有多少人購買
repurchase_count = repurchase_count.groupby(['seller_id']).apply(lambda x:len(x.user_id)).reset_index()
#合併seller_id 與對應的重複購買的人數，並且將沒有重複購買的seller_id的數字填充為0
repurchase_count = pd.merge(pd.DataFrame({'seller_id':range(1,4996,1)}),repurchase_count,how='left',on=['seller_id']).fillna(0)
repurchase_count.rename(columns={0:'repurchase_count'},inplace=True)


In [278]:
repurchase_count.head()

,seller_id,0
0,1,2214.0
1,2,4.0
2,3,2.0
3,4,48.0
4,5,14.0


In [298]:
#統計seller被點擊、加入購物車、購買、收藏的次數
#計算個階段轉換率
seller = user_log[['seller_id','action_type']]
#將action_type 裡面得4種數據進行啞編碼
df  = pd.get_dummies(seller['action_type'],prefix='seller')
#將兩類數具合併並根據seller_id 做 groupby 給予 index 並將
seller = pd.concat([seller,df],axis=1).groupby(['seller_id'],as_index=False).sum()
seller.drop('action_type',axis=1,inplace=True)


In [299]:
seller.columns

Index(['seller_id', 'seller_0', 'seller_1', 'seller_2', 'seller_3'], dtype='object')

In [300]:
#開始計算轉換率，計算標準是 點擊後各加入購物車、購買、收藏的次數等指標的轉換率
seller['seller_purchase_rate'] = round(seller['seller_2'] / seller['seller_0'],5)
seller['seller_addtocart_rate'] = round(seller['seller_1'] / seller['seller_0'],5)
seller['seller_save_rate'] = round(seller['seller_3'] / seller['seller_0'],5)


In [274]:
#統計每一個銷售人被用戶點擊的購買、加入購物車、收藏等次數
buyer_count = user_log[['user_id','seller_id','action_type']]
var = pd.get_dummies(buyer_count['action_type'],prefix='seller_buyer_count')
#將兩張表合併
buyer_count = pd.concat([buyer_count,var],axis=1)
buyer_count.drop('action_type',axis=1,inplace=True)
#將重複的數值刪除
buyer_count.drop_duplicates(inplace=True)
#開始計算每個銷售者個行為的加總
click_count = buyer_count.groupby(['seller_id']).apply(lambda x:x.seller_buyer_count_0.sum())
addtocart_count = buyer_count.groupby(['seller_id']).apply(lambda x:x.seller_buyer_count_1.sum())
buy_count = buyer_count.groupby(['seller_id']).apply(lambda x:x.seller_buyer_count_2.sum())
save_count = buyer_count.groupby(['seller_id']).apply(lambda x:x.seller_buyer_count_3.sum())
#將剛剛計算的表格合併
buyer_count = pd.concat([click_count,addtocart_count,buy_count,save_count],axis=1).reset_index()
buyer_count.rename(columns={0:'click_count',1:'addtocart_count',2:'buy_count',3:'save_count'},inplace=True)


In [301]:
#將對商品種類計算的所有的表進行合併
seller = pd.merge(seller,item_count,on=['seller_id'])
seller = pd.merge(seller,cat_count,on=['seller_id'])
seller = pd.merge(seller,brand_count,on=['seller_id'])
seller = pd.merge(seller,repurchase_count,on=['seller_id'])
seller = pd.merge(seller,buyer_count,on=['seller_id'])



In [302]:
seller.head()

,seller_id,seller_0,seller_1,seller_2,seller_3,seller_purchase_rate,seller_addtocart_rate,seller_save_rate,item_count,cat_count,brand_count,0,click_count,seller_peopleNum_1,buy_count,save_count
0,1,308236.0,444.0,17705.0,12755.0,0.05744,0.00144,0.04138,2977,44,398,2214.0,29251,265,7666,4965
1,2,2030.0,8.0,189.0,144.0,0.09310,0.00394,0.07094,154,10,9,4.0,902,6,161,127
2,3,2399.0,4.0,67.0,175.0,0.02793,0.00167,0.07295,171,4,5,2.0,1103,4,65,150
3,4,2646.0,2.0,294.0,164.0,0.11111,0.00076,0.06198,155,7,6,48.0,1384,2,201,153
4,5,7483.0,9.0,144.0,556.0,0.01924,0.00120,0.07430,660,23,10,14.0,3535,9,120,458


In [303]:
#計算出每個銷售者的商品的數量、種類、品牌佔據總量的比例(表示銷售者整體的市場規模大小)
seller['item_percentage'] = seller['item_count'] / user_log['item_id'].nunique()
seller['cat_percentage'] = seller['cat_count'] / user_log['cat_id'].nunique()
seller['brand_percentage'] = seller['brand_count'] / user_log['brand_id'].nunique()

In [307]:
seller.head()

,seller_id,seller_0,seller_1,seller_2,seller_3,seller_purchase_rate,seller_addtocart_rate,seller_save_rate,item_count,cat_count,brand_count,buyer_count,click_count,addtocart_count,buy_count,save_count,item_percentage,cat_percentage,brand_percentage
0,1,308236.0,444.0,17705.0,12755.0,0.05744,0.00144,0.04138,2977,44,398,2214.0,29251,265,7666,4965,0.002730,0.026538,0.004358
1,2,2030.0,8.0,189.0,144.0,0.09310,0.00394,0.07094,154,10,9,4.0,902,6,161,127,0.000141,0.006031,0.000099
2,3,2399.0,4.0,67.0,175.0,0.02793,0.00167,0.07295,171,4,5,2.0,1103,4,65,150,0.000157,0.002413,0.000055
3,4,2646.0,2.0,294.0,164.0,0.11111,0.00076,0.06198,155,7,6,48.0,1384,2,201,153,0.000142,0.004222,0.000066
4,5,7483.0,9.0,144.0,556.0,0.01924,0.00120,0.07430,660,23,10,14.0,3535,9,120,458,0.000605,0.013872,0.000109


In [308]:
#計算出每用銷售者被點擊、加入購物車、購買與收藏的人數佔整體點擊、加入購物車、購買與收藏的比例
seller['click_buyer_percentage'] = seller['click_count']/user_log[user_log['action_type']==0]['user_id'].nunique()
seller['addtocart_buyer_percentage'] = seller['addtocart_count']/user_log[user_log['action_type']==1]['user_id'].nunique()
seller['buy_buyer_percentage'] = seller['buy_count']/user_log[user_log['action_type']==2]['user_id'].nunique()
seller['save_buyer_percentage'] = seller['save_count']/user_log[user_log['action_type']==3]['user_id'].nunique()

In [310]:
#對數值進行標準化
sell_num_column = seller.columns[seller.dtypes!='uint64']
print(sell_num_column)


Index(['seller_0', 'seller_1', 'seller_2', 'seller_3', 'seller_purchase_rate',
       'seller_addtocart_rate', 'seller_save_rate', 'item_count', 'cat_count',
       'brand_count', 'buyer_count', 'click_count', 'addtocart_count',
       'buy_count', 'save_count', 'item_percentage', 'cat_percentage',
       'brand_percentage', 'click_buyer_percentage',
       'addtocart_buyer_percentage', 'buy_buyer_percentage',
       'save_buyer_percentage'],
      dtype='object')


In [312]:
sell_num_column_mean = seller.loc[:,sell_num_column].mean()
sell_num_column_std = seller.loc[:,sell_num_column].std()
seller.loc[:,sell_num_column] = (seller.loc[:,sell_num_column]-sell_num_column_mean)/sell_num_column_std
seller.head()

,seller_id,seller_0,seller_1,seller_2,seller_3,seller_purchase_rate,seller_addtocart_rate,seller_save_rate,item_count,cat_count,...,addtocart_count,buy_count,save_count,item_percentage,cat_percentage,brand_percentage,click_buyer_percentage,addtocart_buyer_percentage,buy_buyer_percentage,save_buyer_percentage
0,1,10.972787,11.897055,14.249959,7.235092,-0.120234,-0.051657,-0.053750,8.107215,1.797897,...,9.703700,10.099788,6.360937,8.107215,1.797897,8.971108,6.739658,9.703700,10.099788,6.360937
1,2,-0.282662,-0.204431,-0.392982,-0.272506,-0.018504,0.119926,-0.013843,-0.188952,-0.269604,...,-0.248208,-0.393341,-0.372739,-0.188952,-0.269604,-0.287303,-0.444305,-0.248208,-0.393341,-0.372739
2,3,-0.269099,-0.315454,-0.494971,-0.254051,-0.204419,-0.035871,-0.011130,-0.138993,-0.634457,...,-0.325057,-0.527563,-0.340727,-0.138993,-0.634457,-0.382505,-0.393370,-0.325057,-0.527563,-0.340727
3,4,-0.260019,-0.370965,-0.305204,-0.260600,0.032874,-0.098327,-0.025940,-0.186014,-0.452031,...,-0.401906,-0.337414,-0.336552,-0.186014,-0.452031,-0.358704,-0.322161,-0.401906,-0.337414,-0.336552
4,5,-0.082222,-0.176675,-0.430601,-0.027234,-0.229209,-0.068129,-0.009307,1.298069,0.520911,...,-0.132935,-0.450665,0.087956,1.298069,0.520911,-0.263502,0.222927,-0.132935,-0.450665,0.087956


In [313]:
#保存商品特徵
seller_file_path = 'seller_info_features.pkl'
pickle.dump(seller,open(seller_file_path,'wb'))

In [314]:
seller_file_path = 'seller_info_features'
if os.path.exists(seller_file_path):
    seller = pickle.load(open(seller_file_path,'r'))

In [315]:
#先將目前所做好的資料合併到測試與訓練集的數據中
train_data = pd.read_csv('train_format1.csv')
train_data.rename(columns={'merchant_id':'seller_id'},inplace=True)
test_data = pd.read_csv('test_format1.csv')
test_data.rename(columns={'merchant_id':'seller_id'},inplace=True)
all_data = pd.concat([train_data[['user_id','seller_id']],test_data[['user_id','seller_id']]],ignore_index=True)
data_all = pd.merge(all_data,user_log,on=['user_id','seller_id'])



In [385]:
#計算消費者對預測的商店的各種行為的特徵，像是點擊、加入購物車、購買、收藏總次數，以及上述提及過的轉換率
df = data_all[['user_id','seller_id','action_type']]
df_all = pd.get_dummies(df['action_type'],prefix='user_seller_action')
df = pd.concat([df,df_all],axis=1).groupby(['user_id','seller_id'],as_index=False).sum()

In [386]:
#計算消費者對預測商店點擊的總天數，意思是目前合併的數據是將 預測+訓練的數據 合併再一起，再將原先有的user_log數據
#根據user_id與seller_id用groupby方式整併到新的資料內，所以目前要做的是用原先user_log資料來分預測+訓練資量
#所以要算的預測+訓練的目前每個人對於每個店家，點擊、加入購物車、購買、收藏有多少次
click_days = data_all[data_all.action_type==0]
click_days = click_days[['user_id','seller_id','time_stamp','action_type']]
click_days = click_days.groupby(['user_id','seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
click_days.rename(columns={0:'click_days'},inplace=True)
df = pd.merge(df,click_days,how='left',on=['user_id','seller_id'])
df.click_days.fillna(0,inplace=True)

In [387]:
#計算消費者對預測商店加入購物車的總天數
addtocart_days = data_all[data_all.action_type==1]
addtocart_days = addtocart_days[['user_id','seller_id','time_stamp','action_type']]
addtocart_days = addtocart_days.groupby(['user_id','seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
addtocart_days.rename(columns={0:'addtocart_days'},inplace=True)
df = pd.merge(df,addtocart_days,how='left',on=['user_id','seller_id'])
df.addtocart_days.fillna(0,inplace=True)


In [388]:
#計算消費者對預測商店購買的總天數
purchase_days = data_all[data_all.action_type==2]
purchase_days = purchase_days[['user_id','seller_id','time_stamp','action_type']]
purchase_days = purchase_days.groupby(['user_id','seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
purchase_days.rename(columns={0:'purchase_days'},inplace=True)
df = pd.merge(df,purchase_days,how='left',on=['user_id','seller_id'])
df.purchase_days.fillna(0,inplace=True)


In [389]:
#計算消費者對預測商店收藏的總天數
save_days = data_all[data_all.action_type==3]
save_days = save_days[['user_id','seller_id','time_stamp','action_type']]
save_days = save_days.groupby(['user_id','seller_id']).apply(lambda x:x.time_stamp.nunique()).reset_index()
save_days.rename(columns={0:'save_days'},inplace=True)
df = pd.merge(df,save_days,how='left',on=['user_id','seller_id'])
df.save_days.fillna(0,inplace=True)

In [391]:
#點擊商品數量
item_count = data_all[['user_id','seller_id','item_id','action_type']]
item_count_click = item_count[item_count.action_type==0]
item_click_final = item_count_click.groupby(['user_id','seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_click_final.rename(columns={0:'item_click_count'},inplace=True)
df=pd.merge(df,item_click_final,how='left',on=['user_id','seller_id'])
df.item_click_count.fillna(0,inplace=True)

In [394]:
#加入購物車商品數量
item_count_addtocart = item_count[item_count.action_type==1]
item_addtocart_final = item_count_addtocart.groupby(['user_id','seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_addtocart_final.rename(columns={0:'item_addtocart_count'},inplace=True)
df=pd.merge(df,item_addtocart_final,how='left',on=['user_id','seller_id'])
df.item_addtocart_count.fillna(0,inplace=True)

In [395]:
#購買商品數量
item_count_purchase = item_count[item_count.action_type==2]
item_pruchase_final = item_count_purchase.groupby(['user_id','seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_pruchase_final.rename(columns={0:'item_purchase_count'},inplace=True)
df=pd.merge(df,item_pruchase_final,how='left',on=['user_id','seller_id'])
df.item_purchase_count.fillna(0,inplace=True)

In [400]:
#收藏商品數量
item_count_save = item_count[item_count.action_type==3]
item_save_final = item_count_save.groupby(['user_id','seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
item_save_final.rename(columns={0:'item_save_count'},inplace=True)
df=pd.merge(df,item_save_final,how='left',on=['user_id','seller_id'])
df.item_save_count.fillna(0,inplace=True)

In [ ]:
#點擊種類數量

In [403]:
cat_count =data_all[['user_id','seller_id','cat_id','action_type']]

In [408]:
cat_count_click = cat_count[cat_count.action_type==0]
cat_click_final = cat_count_click.groupby(['user_id','seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_click_final.rename(columns={0:'cat_click_count'},inplace=True)
df=pd.merge(df,cat_click_final,how='left',on=['user_id','seller_id'])
df.cat_click_count.fillna(0,inplace=True)

In [409]:
df.head()

,user_id,seller_id,action_type,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,click_days,addtocart_days,purchase_days,save_days,item_click_count,item_addtocart_count,item_purchase_count,item_save_count,cat_click_count
0,1,1019,8.0,10.0,0.0,4.0,0.0,1.0,0.0,1,0.0,1.0,0.0,1,0.0,1.0
1,2,1679,6.0,0.0,0.0,3.0,0.0,0.0,0.0,1,0.0,0.0,0.0,2,0.0,0.0
2,2,1784,4.0,0.0,0.0,2.0,0.0,0.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0
3,3,2313,5.0,4.0,0.0,1.0,1.0,1.0,0.0,1,1.0,2.0,0.0,1,1.0,2.0
4,4,1186,2.0,4.0,0.0,1.0,0.0,1.0,0.0,1,0.0,1.0,0.0,1,0.0,1.0


In [ ]:
#加入購物車種類數量

In [410]:
cat_count_addtocart = cat_count[cat_count.action_type==1]
cat_addtocart_final = cat_count_addtocart.groupby(['user_id','seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_addtocart_final.rename(columns={0:'cat_addtocart_count'},inplace=True)
df=pd.merge(df,cat_addtocart_final,how='left',on=['user_id','seller_id'])
df.cat_addtocart_count.fillna(0,inplace=True)

In [411]:
#購買種類數量
cat_count_purchase = cat_count[cat_count.action_type==2]
cat_purchase_final = cat_count_purchase.groupby(['user_id','seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_purchase_final.rename(columns={0:'cat_purchase_count'},inplace=True)
df=pd.merge(df,cat_purchase_final,how='left',on=['user_id','seller_id'])
df.cat_purchase_count.fillna(0,inplace=True)

In [413]:
#收藏種類數量
cat_count_save = cat_count[cat_count.action_type==3]
cat_save_final = cat_count_save.groupby(['user_id','seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
cat_save_final.rename(columns={0:'cat_save_count'},inplace=True)
df=pd.merge(df,cat_save_final,how='left',on=['user_id','seller_id'])
df.cat_save_count.fillna(0,inplace=True)

In [412]:
df.head()

,user_id,seller_id,action_type,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,click_days,addtocart_days,purchase_days,save_days,item_click_count,item_addtocart_count,item_purchase_count,item_save_count,cat_click_count,cat_addtocart_count,cat_purchase_count
0,1,1019,8.0,10.0,0.0,4.0,0.0,1.0,0.0,1,0.0,1.0,0.0,1,0.0,1.0,0.0,1
1,2,1679,6.0,0.0,0.0,3.0,0.0,0.0,0.0,1,0.0,0.0,0.0,2,0.0,0.0,0.0,1
2,2,1784,4.0,0.0,0.0,2.0,0.0,0.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,1
3,3,2313,5.0,4.0,0.0,1.0,1.0,1.0,0.0,1,1.0,2.0,0.0,1,1.0,2.0,0.0,1
4,4,1186,2.0,4.0,0.0,1.0,0.0,1.0,0.0,1,0.0,1.0,0.0,1,0.0,1.0,0.0,1


In [414]:
brand_count =data_all[['user_id','seller_id','brand_id','action_type']]

In [416]:
#點擊品牌數量
brand_count_click = brand_count[brand_count.action_type==0]
brand_click_final = brand_count_click.groupby(['user_id','seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_click_final.rename(columns={0:'brand_click_count'},inplace=True)
df= pd.merge(df,brand_click_final,how='left',on=['user_id','seller_id'])
df.brand_click_count.fillna(0,inplace=True)

In [428]:
#加入購物車品牌數量
brand_count_addtocart = brand_count[brand_count.action_type==1]
brand_addtocart_final = brand_count_addtocart.groupby(['user_id','seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_addtocart_final.rename(columns={0:'brand_addtocart_count'},inplace=True)
df= pd.merge(df,brand_addtocart_final,how='left',on=['user_id','seller_id'])
df.brand_addtocart_count.fillna(0,inplace=True)

In [429]:
#購買品牌數量
brand_count_purchase = brand_count[brand_count.action_type==2]
brand_purchase_final = brand_count_purchase.groupby(['user_id','seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_purchase_final.rename(columns={0:'brand_purchase_count'},inplace=True)
df= pd.merge(df,brand_purchase_final,how='left',on=['user_id','seller_id'])
df.brand_purchase_count.fillna(0,inplace=True)

In [430]:
#收藏品牌數量
brand_count_save = brand_count[brand_count.action_type==3]
brand_save_final = brand_count_save.groupby(['user_id','seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
brand_save_final.rename(columns={0:'brand_save_count'},inplace=True)
df= pd.merge(df,brand_save_final,how='left',on=['user_id','seller_id'])
df.brand_save_count.fillna(0,inplace=True)

In [434]:
#觀察到puchase 的指標都是 Int 類型數值，統一轉換成float 以利後續操作
for i in ['item_purchase_count','cat_purchase_count','cat_purchase_count','brand_purchase_count']:
    df[i] =df[i].astype('float64')

#對數值型特徵標準化
user_seller_num_column = df.columns[df.dtypes=='float64']
print(user_seller_num_column)

Index(['action_type', 'user_seller_action_0', 'user_seller_action_1',
       'user_seller_action_2', 'user_seller_action_3', 'click_days',
       'addtocart_days', 'save_days', 'item_click_count',
       'item_addtocart_count', 'item_purchase_count', 'item_save_count',
       'cat_click_count', 'cat_addtocart_count', 'cat_purchase_count',
       'cat_save_count', 'brand_click_count', 'brand_addtocart_count',
       'brand_purchase_count', 'brand_save_count'],
      dtype='object')


In [435]:
user_seller_num_column_mean = df.loc[:,user_seller_num_column].mean()
user_seller_num_column_std = df.loc[:,user_seller_num_column].std()
df.loc[:,user_seller_num_column] = (df.loc[:,user_seller_num_column]-user_seller_num_column_mean)/user_seller_num_column_std
df.head()

,user_id,seller_id,action_type,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,click_days,addtocart_days,purchase_days,...,item_purchase_count,item_save_count,cat_click_count,cat_addtocart_count,cat_purchase_count,cat_save_count,brand_click_count,brand_addtocart_count,brand_purchase_count,brand_save_count
0,1,1019,0.846737,0.048748,-0.113497,3.113534,-0.264393,-0.377563,-0.129999,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
1,2,1679,0.437302,-0.487477,-0.113497,1.944228,-0.264393,-0.967207,-0.129999,1,...,1.028622,-0.271276,-0.897370,-0.125234,-0.293369,-0.368967,-1.817830,-0.117466,-0.094,-0.466480
2,2,1784,0.027867,-0.487477,-0.113497,0.774921,-0.264393,-0.967207,-0.129999,1,...,-0.348307,-0.271276,-0.897370,-0.125234,-0.293369,-0.368967,-1.817830,-0.117466,-0.094,-0.466480
3,3,2313,0.232585,-0.272987,-0.113497,-0.394386,0.416284,-0.377563,-0.129999,1,...,-0.348307,0.465957,0.145228,-0.125234,-0.293369,1.076768,0.096026,-0.117466,-0.094,2.007016
4,4,1186,-0.381568,-0.272987,-0.113497,-0.394386,-0.264393,-0.377563,-0.129999,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480


In [436]:
#保存特徵
user_seller_path ='user_seller_action.pkl'
pickle.dump(df,open(user_seller_path,'wb'))

In [437]:
#讀取用戶特徵
user_seller_path ='user_seller_action.pkl'
if os.path.exists(user_seller_path):
    df = pickle.load(open(user_seller_path,'rb'))

In [440]:
def build_train():
    file_path = 'train_data_features.pkl'
    if os.path.exists(file_path):
        train_data = pickle.load(open(file_path,'rb'))
    else:
        train_data  = pd.read_csv('train_format1.csv')
        train_data.rename(columns={'merchant_id':'seller_id'},inplace=True)
        user_info = pickle.load(open('user_info_features.pkl','rb'))
        train_data = pd.merge(train_data,user_info,how='left',on=['user_id'])
        seller_info = pickle.load(open('seller_info_features.pkl','rb'))
        train_data = pd.merge(train_data,seller_info,how='left',on=['seller_id'])
        user_seller_info = pickle.load(open('user_seller_action.pkl','rb'))
        train_data = pd.merge(train_data,user_seller_info,how='left',on=['user_id','seller_id'])
        del user_info,seller_info,user_seller_info
        pickle.dump(train_data,open(file_path,'wb'))
    return train_data
train_data = build_train()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260864 entries, 0 to 260863
Columns: 117 entries, user_id to brand_save_count
dtypes: float64(100), int64(5), uint8(12)
memory usage: 214.0 MB


In [442]:
train_data.head()

,user_id,seller_id,label,age_0.0,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,...,item_purchase_count,item_save_count,cat_click_count,cat_addtocart_count,cat_purchase_count,cat_save_count,brand_click_count,brand_addtocart_count,brand_purchase_count,brand_save_count
0,34176,3906,0,0,0,0,0,0,0,1,...,-0.348307,1.203190,1.709124,-0.125234,-0.293369,2.522503,0.096026,-0.117466,-0.094,2.007016
1,34176,121,0,0,0,0,0,0,0,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
2,34176,4356,1,0,0,0,0,0,0,1,...,1.028622,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
3,34176,2217,0,0,0,0,0,0,0,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
4,230784,4818,0,1,0,0,0,0,0,0,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480


In [443]:
def build_test():
    file_path_test = 'test_data_features.pkl'
    if os.path.exists(file_path_test):
        test_data = pickle.load(open(file_path_test,'rb'))
    else:
        test_data  = pd.read_csv('test_format1.csv')
        test_data.rename(columns={'merchant_id':'seller_id'},inplace=True)
        user_info = pickle.load(open('user_info_features.pkl','rb'))
        test_data = pd.merge(test_data,user_info,how='left',on=['user_id'])
        seller_info = pickle.load(open('seller_info_features.pkl','rb'))
        test_data = pd.merge(test_data,seller_info,how='left',on=['seller_id'])
        user_seller_info = pickle.load(open('user_seller_action.pkl','rb'))
        test_data = pd.merge(test_data,user_seller_info,how='left',on=['user_id','seller_id'])
        del user_info,seller_info,user_seller_info
        pickle.dump(test_data,open(file_path_test,'wb'))
    return test_data
test_data = build_test()

In [444]:
test_data.head()

,user_id,seller_id,prob,age_0.0,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,...,item_purchase_count,item_save_count,cat_click_count,cat_addtocart_count,cat_purchase_count,cat_save_count,brand_click_count,brand_addtocart_count,brand_purchase_count,brand_save_count
0,163968,4605,NaN,1,0,0,0,0,0,0,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
1,360576,1581,NaN,0,0,1,0,0,0,0,...,5.159411,-0.271276,1.187825,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
2,98688,1964,NaN,0,0,0,0,0,0,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
3,98688,3645,NaN,0,0,0,0,0,0,1,...,-0.348307,-0.271276,-0.376071,-0.125234,-0.293369,-0.368967,0.096026,-0.117466,-0.094,-0.466480
4,295296,3361,NaN,0,0,1,0,0,0,0,...,-0.348307,1.203190,1.187825,-0.125234,-0.293369,1.076768,2.009882,-0.117466,-0.094,2.007016


In [445]:
from sklearn.model_selection import train_test_split
#只要會以70%訓練集、30%測試集，測試集當中15%給驗證集
x= train_data.loc[:,train_data.columns != 'label']
y= train_data.loc[:,train_data.columns == 'label']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=100)
del xtrain['user_id']
del xtrain['seller_id']
del xtest['user_id']
del xtest['seller_id']
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)

(182604, 114) (182604, 1) (78260, 114) (78260, 1)


In [446]:
#先用最簡單的邏輯回歸模型測試
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.linear_model import LogisticRegression

In [448]:
params = [{'penalty':['l1'],'C':[100,1500],'solver':['liblinear']},{'penalty':['l2'],'C':[100,1500],'solver':['lbfgs']}]
clf = LogisticRegression(random_state=100,max_iter=500,verbose=2)
log_gridcv =GridSearchCV(clf,params,scoring='roc_auc',cv=10,verbose=2)
log_gridcv.fit(xtrain,ytrain)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] C=100, penalty=l1, solver=liblinear .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\austin\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[LibLinear][CV] .............. C=100, penalty=l1, solver=liblinear, total= 2.7min
[CV] C=100, penalty=l1, solver=liblinear .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s
C:\Users\austin\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[LibLinear][CV] .............. C=100, penalty=l1, solver=liblinear, total= 2.3min
[CV] C=100, penalty=l1, solver=liblinear .............................


...

C:\Users\austin\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\austin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.0s finished
[

[CV] ................. C=1500, penalty=l2, solver=lbfgs, total=  28.6s


C:\Users\austin\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[LibLinear]

GridSearchCV(cv=10,
             estimator=LogisticRegression(max_iter=500, random_state=100,
                                          verbose=2),
             param_grid=[{'C': [100, 1500], 'penalty': ['l1'],
                          'solver': ['liblinear']},
                         {'C': [100, 1500], 'penalty': ['l2'],
                          'solver': ['lbfgs']}],
             scoring='roc_auc', verbose=2)

In [449]:
print(log_gridcv.best_score_)    #查看最佳分数(此处为neg_mean_absolute_error)
print(log_gridcv.best_params_)   #查看最佳参数
print(log_gridcv.cv_results_)
print(log_gridcv.best_estimator_) 
lr=log_gridcv.best_estimator_

0.6335802271076878
{'C': 1500, 'penalty': 'l1', 'solver': 'liblinear'}
{'mean_fit_time': array([143.67509727, 102.97083738,  26.37671065,  25.65103478]), 'std_fit_time': array([15.03661344,  4.91428707,  0.84917842,  1.03535991]), 'mean_score_time': array([0.06126814, 0.04107702, 0.03665509, 0.03847957]), 'std_score_time': array([0.0508109 , 0.00540877, 0.00454192, 0.0112712 ]), 'param_C': masked_array(data=[100, 1500, 100, 1500],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_penalty': masked_array(data=['l1', 'l1', 'l2', 'l2'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_solver': masked_array(data=['liblinear', 'liblinear', 'lbfgs', 'lbfgs'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1500, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 100, 'pe

In [463]:
test_data1 = test_data.copy()

In [541]:
x_train, y_train = train_data.drop(['label'], axis=1), train_data['label']

In [542]:
train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, test_size=0.2, random_state=100)

In [543]:
def xgboost_train(train_x, train_y, valid_x, valid_y, verbose=True):
    xgboost_model = xgb.XGBClassifier(
        max_depth=20
        ,n_estimators=1500
        ,min_child_weight=400
        ,colsample_bytree=0.5
        ,subsample=0.7
        ,eta=0.05
        ,seed=42        )

    xgboost_model.fit(
        train_x 
        ,train_y
        ,eval_metric='auc'
        ,eval_set=[(train_x, train_y), (valid_x, valid_y)]
        ,verbose=verbose
        ,early_stopping_rounds=20
    )
    print(xgboost_model.best_score)
    return xgboost_model

In [544]:
import xgboost as xgb
xgboost_model = xgboost_train(train_x, train_y, valid_x, valid_y, verbose=False)

0.680656


In [545]:
result = test_data.copy()

In [470]:
#將模型運用到測試集當中，並且輸出呈excel檔案
xgb_probability = model_xgb.predict_proba(test_data1)

result['prob'] = pd.Series(xgb_probability[:,1])
result.to_csv('xgb_model_result.csv', index=False)

In [514]:

#給予lightgbm參數，並且訓練她
def lightgbm_train(train_x, train_y, valid_x, valid_y, verbose=True):
    lightgbm_model = lgb.LGBMClassifier(
        max_depth=20
        ,n_estimators=1000
        ,min_child_weight=200 
        ,colsample_bytree=0.5 
        ,subsample=0.7 
        ,eta=0.05    
        ,seed=42        
    )

    lightgbm_model.fit(
        train_x, 
        train_y,
        eval_metric='auc',
        eval_set=[(train_x, train_y), (valid_x, valid_y)],
        verbose=verbose,
        early_stopping_rounds=20
    )

    print(lightgbm_model.best_score_['valid_1']['auc'])
    return lightgbm_model

In [515]:

import lightgbm as lgb
lightgbm_model = lightgbm_train(train_x, train_y, valid_x, valid_y, verbose=False)

0.6905247102012406


In [546]:
#將模型運用到測試集當中，並且輸出呈excel檔案
lightgbm_probability = model_lgb.predict_proba(test_data1)
result['prob'] = pd.Series(lightgbm_probability[:,1])
result.to_csv('lightgbm_model_result.csv', index=False)

In [479]:
import catboost as cat
def catboost_model_train(train_x, train_y, valid_x, valid_y, verbose=True):
    catboost_model = cat.CatBoostClassifier(
        learning_rate=0.02
        ,iterations=6000
        ,eval_metric='AUC'
        ,od_wait=60
        ,od_type='Iter'
        ,random_state=10
        ,thread_count=7
        ,l2_leaf_reg=1, verbose=verbose)
    
    catboost_model.fit(
        train_x
        ,train_y
        ,eval_set=[(valid_x, valid_y)]
        ,early_stopping_rounds=50
        ,use_best_model=True)

    print(catboost_model.best_score_['validation']['AUC'])
    return catboost_model

In [480]:
train_x, valid_x, train_y, valid_y
catboost_model = catboost_model_train(train_x, train_y, valid_x, valid_y, verbose=False)


0.6890287432162316


In [481]:
#將模型運用到測試集當中，並且輸出呈excel檔案
catboost_prob = model_cat.predict_proba(test_data1)
result['prob'] = pd.Series(catboost_prob[:,1])
result.to_csv('catboost_model_result.csv', index=False)

In [492]:
xgb_6819 =pd.read_csv('submission_xgb.csv')
cat_6890 =pd.read_csv('submission_cat.csv')
lgb_6871 =pd.read_csv('submission_lgb.csv')

In [494]:
df = np.array([lgb_6871.prob, xgb_6819.prob, cat_6890.prob])
np.corrcoef(df)

array([[1.        , 0.94209482, 0.93599242],
       [0.94209482, 1.        , 0.90758663],
       [0.93599242, 0.90758663, 1.        ]])

In [495]:
result=xgb_6819.copy()

In [499]:
result.prob = 0.5*lgb_6871.prob+0.3*cat_6890.prob+0.2*xgb_6819.prob
result.to_csv('blending1.csv',index=False)

In [501]:
df1= pd.read_csv('blending1.csv')